In [1]:
import sys
from Load import *
from UE import *
from Satellite import *
from Oracle import *
import time
import os


def initial_assignment(UEss, Satss, oracle):
    C = UEss[0].coverage_info
    if oracle is None:
        # random assign
        for ueid in UEss:
            ue = UEss[ueid]
            possible_satellites = np.where(C[ueid, :, 0] == 1)[0]
            assert (len(possible_satellites) > 0)
            satid = random.choice(possible_satellites)
            ue.serving_satellite = Satss[satid]
            ue.serving_satellite_history.append(satid)
    else:
        for ueid in UEss:
            ue = UEss[ueid]
            satid = oracle.query_init_satellite(ueid)
            assert (C[ueid, satid, 0] == 1)
            ue.serving_satellite = Satss[satid]
            ue.serving_satellite_history.append(satid)


def monitor_timestamp(env):
    while True:
        if env.now % 100 == 0:
            print(f"Current simulation time {env.now}", file=sys.stderr)
        yield env.timeout(1)


if __name__ == "__main__":
    # configuration
    # When oracle_simulation is False, the oracle_assignment could be True/False
    # When oracle_simulation is True, the oracle_assignment must be True
    if oracle_simulation:
        assert (oracle_assignment)
    random.seed(10)

    # Loading scenarios
    beginning = time.time()
    UEs_template, satellites_template, C = load_scenario()
    DURATION = C.shape[2]
    env = simpy.Environment()

    assignment_oracle = None
    simulation_oracle = None
    if oracle_assignment:
        assignment_oracle = Oracle()
    if oracle_simulation:
        simulation_oracle = assignment_oracle

    UEs = {}
    for ue_template in UEs_template:
        ID = ue_template.ID
        if ID not in UEs:
            UEs[ID] = UE(
                identity=ID,
                position_x=ue_template.x,
                position_y=ue_template.y,
                coverage_info=C,
                oracle=simulation_oracle,
                env=env
            )
        else:
            print("ERROR")

    Satellites = {}
    for sat_template in satellites_template:
        ID = sat_template.ID
        if ID not in Satellites:
            Satellites[ID] = Satellite(
                identity=ID,
                position_x=sat_template.x,
                position_y=sat_template.y,
                height=sat_template.h,
                coverage_r=sat_template.r,
                velocity=sat_template.v,
                sind=sat_template.sind,
                cosd=sat_template.cosd,
                coverage_info=C,
                max_access_opportunity=max_access_opportunity,
                max_access_slots=max_access_slots,
                env=env,
                oracle=simulation_oracle,
            )
        else:
            print("ERROR")

    for ueid in UEs:
        UEs[ueid].satellites = Satellites

    for satid in Satellites:
        Satellites[satid].UEs = UEs
        Satellites[satid].satellites = Satellites

    print(f"Loading scenario in the simulation takes {time.time() - beginning}")
    # ========= UE, SAT objectives are ready =========
    initial_assignment(UEs, Satellites, assignment_oracle)

    env.process(monitor_timestamp(env))

    print('============= Experiment Log =============')
    env.run(until=DURATION)
    print('============= Experiment Ends =============')
    counter = allCounters(Satellites, UEs)
    counter.give_result()


Loading scenario in the simulation takes 0.08807492256164551
============= Experiment Log =============
(4, 7, 2)

(3310, 3234, 3062)
(5, 7, 2)

(4534, 3386, 3165)
(5, 7, 2)

(4482, 3374, 3148)
(5, 7, 2)

(4384, 3277, 3060)
(5, 7, 2)

(4318, 3234, 3018)
(6, 9, 5)

(3393, 3330, 2831)
(6, 9, 5)

(3341, 3330, 2819)
(6, 9, 5)

(3234, 3178, 2704)
(6, 9, 5)

(3234, 3097, 2664)


Current simulation time 0
Current simulation time 100
Current simulation time 200
Current simulation time 300
Current simulation time 400
Current simulation time 500
Current simulation time 600
Current simulation time 700
Current simulation time 800
Current simulation time 900
Current simulation time 1000
Current simulation time 1100
Current simulation time 1200
Current simulation time 1300
Current simulation time 1400
Current simulation time 1500
Current simulation time 1600
Current simulation time 1700
Current simulation time 1800
Current simulation time 1900
Current simulation time 2000
Current simulation time 2100
Current simulation time 2200
Current simulation time 2300
Current simulation time 2400
Current simulation time 2500
Current simulation time 2600
Current simulation time 2700
Current simulation time 2800
Current simulation time 2900


(8, 6, 5)

(5038, 3235, 3063)
(8, 6, 5)

(4980, 3234, 3070)
(10, 13, 8)

(3804, 3388, 3330)
(10, 13, 8)

(3720, 3397, 3330)
(10, 13, 8)

(3681, 3402, 3329)
(10, 13, 8)

(3499, 3235, 3158)
(10, 13, 8)

(3506, 3235, 3164)


Current simulation time 3000
Current simulation time 3100
Current simulation time 3200
Current simulation time 3300
Current simulation time 3400
Current simulation time 3500
Current simulation time 3600
Current simulation time 3700
Current simulation time 3800
Current simulation time 3900
Current simulation time 4000
Current simulation time 4100
Current simulation time 4200
Current simulation time 4300
Current simulation time 4400


(13, 8, 10)

(3714, 3594, 3423)
(13, 8, 10)

(3691, 3560, 3423)
(13, 8, 10)

(3596, 3473, 3422)
(13, 8, 10)

(3450, 3330, 3222)
(15, 12, 10)

(3829, 3797, 2830)
(12, 15, 10)

(3805, 3763, 2839)
(15, 12, 10)

(3841, 3756, 2769)
(15, 12, 10)

(3804, 3765, 2780)
(13, 8, 10)

(3234, 3109, 3079)
(11, 13, 8)

(4530, 3234, 3113)
(11, 13, 8)

(4506, 3234, 3114)
(11, 13, 8)

(4529, 3234, 3108)
(11, 13, 8)

(4530, 3234, 3103)


...

(29, 30, 28)

(3824, 3234, 2304)
(34, 32, 37)

(10000000.0, 3559, 3235)
(34, 36, 32)

(10000000.0, 10000000.0, 3330)
(34, 36, 32)

(10000000.0, 10000000.0, 3330)
(34, 36, 32)

(10000000.0, 10000000.0, 3329)
(34, 36, 32)

(10000000.0, 10000000.0, 3330)
(34, 36, 32)

(10000000.0, 10000000.0, 3330)
(34, 36, 32)

(10000000.0, 10000000.0, 3329)
(34, 36, 32)

(10000000.0, 10000000.0, 3329)
(34, 36, 32)

(10000000.0, 10000000.0, 3330)


Current simulation time 15800
Current simulation time 15900
Current simulation time 16000
Current simulation time 16100
Current simulation time 16200
Current simulation time 16300
Current simulation time 16400
Current simulation time 16500
Current simulation time 16600
Current simulation time 16700
Current simulation time 16800
Current simulation time 16900
Current simulation time 17000
Current simulation time 17100
Current simulation time 17200


(32, 34, 37)

(10000000.0, 10000000.0, 10000000.0)
(32, 34, 37)

(10000000.0, 10000000.0, 10000000.0)
(32, 34, 37)

(10000000.0, 10000000.0, 10000000.0)
(32, 34, 37)

(10000000.0, 10000000.0, 10000000.0)
(32, 34, 37)

(10000000.0, 10000000.0, 10000000.0)
(32, 34, 37)

(10000000.0, 10000000.0, 10000000.0)
(32, 34, 37)

(10000000.0, 10000000.0, 10000000.0)
(32, 34, 36)

(10000000.0, 10000000.0, 10000000.0)
(32, 34, 36)

(10000000.0, 10000000.0, 10000000.0)
(32, 34, 36)

(10000000.0, 10000000.0, 10000000.0)
(32, 34, 36)

(10000000.0, 10000000.0, 10000000.0)


Current simulation time 17300
Current simulation time 17400
Current simulation time 17500
Current simulation time 17600
Current simulation time 17700
Current simulation time 17800
Current simulation time 17900
Current simulation time 18000
Current simulation time 18100
Current simulation time 18200
Current simulation time 18300
Current simulation time 18400
Current simulation time 18500
Current simulation time 18600
Current simulation time 18700
Current simulation time 18800
Current simulation time 18900
Current simulation time 19000
Current simulation time 19100
Current simulation time 19200
Current simulation time 19300
Current simulation time 19400
Current simulation time 19500
Current simulation time 19600
Current simulation time 19700
Current simulation time 19800
Current simulation time 19900


(34, 35, 36)

(10000000.0, 10000000.0, 10000000.0)
============= Experiment Ends =============


In [2]:
counter.give_result(100)

Total signalling: 1939
Total handover: 277
removed 398901 elements


In [2]:
[1,2,3][:2]

[1, 2]

In [ ]:
Loading scenario in the simulation takes 0.07620716094970703
============= Experiment Log =============
[0, 2, 3, 4, 7]

[1237, 3062, 1127, 3310, 3234]
[0, 2, 4, 5, 7]

[1469, 3165, 3042, 4534, 3386]
[0, 2, 4, 5, 7]

[1384, 3148, 2876, 4482, 3374]
[0, 2, 4, 5, 7]

[1306, 3060, 2856, 4384, 3277]